## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Jamestown,42.0970,-79.2353,clear sky,36.99,87,1,3.00,US,2021-12-05 03:04:49
1,1,Bengkulu,-3.8004,102.2655,overcast clouds,77.85,84,100,6.82,ID,2021-12-05 03:06:09
2,2,San Carlos De Bariloche,-41.1456,-71.3082,clear sky,54.91,57,0,3.62,AR,2021-12-05 03:06:10
3,3,Sari,36.5633,53.0601,clear sky,46.58,47,0,4.23,IR,2021-12-05 03:06:10
4,4,Biscarrosse,44.3945,-1.1672,broken clouds,48.15,81,75,14.97,FR,2021-12-05 03:06:10


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input(\
    "What is the minimum temperature you would like for your trip? "))
max_temp = float(input(\
    "What is the maximum temperature you would like for your trip? "))

In [37]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df['Max Temp']<=max_temp) & \
    (city_data_df['Max Temp']>=min_temp)]
preferred_cities_df

,City_ID,City,Lat,Lng,Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Bengkulu,-3.8004,102.2655,overcast clouds,77.85,84,100,6.82,ID,2021-12-05 03:06:09
2,2,San Carlos De Bariloche,-41.1456,-71.3082,clear sky,54.91,57,0,3.62,AR,2021-12-05 03:06:10
7,7,Bredasdorp,-34.5322,20.0403,moderate rain,59.29,95,100,17.83,ZA,2021-12-05 03:06:11
8,8,Rikitea,-23.1203,-134.9692,overcast clouds,76.17,72,89,8.79,PF,2021-12-05 03:06:12
10,10,Kita,13.0349,-9.4895,scattered clouds,71.67,21,48,4.38,ML,2021-12-05 03:06:12
...,...,...,...,...,...,...,...,...,...,...,...
696,696,Monrovia,6.3005,-10.7969,light rain,79.27,82,21,6.04,LR,2021-12-05 03:25:08
701,701,Mamou,10.5000,-12.0000,scattered clouds,69.33,31,39,3.13,GN,2021-12-05 03:25:10
703,703,Edgewater,28.9889,-80.9023,broken clouds,62.98,100,75,0.00,US,2021-12-05 03:23:29
707,707,Machico,32.7000,-16.7667,broken clouds,65.48,72,75,13.80,PT,2021-12-05 03:25:12


In [48]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.count())



City_ID        340
City           340
Lat            340
Lng            340
Description    340
Max Temp       340
Humidity       340
Cloudiness     340
Wind Speed     340
Country        338
Date           340
dtype: int64


In [47]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df[preferred_cities_df['Country'].notna()]
print(clean_df.count())


City_ID        338
City           338
Lat            338
Lng            338
Description    338
Max Temp       338
Humidity       338
Cloudiness     338
Wind Speed     338
Country        338
Date           338
dtype: int64


In [49]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,77.85,overcast clouds,-3.8004,102.2655,
2,San Carlos De Bariloche,AR,54.91,clear sky,-41.1456,-71.3082,
7,Bredasdorp,ZA,59.29,moderate rain,-34.5322,20.0403,
8,Rikitea,PF,76.17,overcast clouds,-23.1203,-134.9692,
10,Kita,ML,71.67,scattered clouds,13.0349,-9.4895,
12,Mataura,NZ,51.48,light rain,-46.1927,168.8643,
18,Maldonado,UY,63.79,overcast clouds,-34.6667,-54.9167,
19,Batemans Bay,AU,64.20,overcast clouds,-35.7167,150.1833,
20,Ribeira Grande,PT,63.77,scattered clouds,38.5167,-28.7000,
21,Rio Claro,BR,63.81,broken clouds,-22.4114,-47.5614,


In [50]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng=row['Lng']
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index, "Hotel Name"] = "No Hotel Found"
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [53]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df['Hotel Name']!='No Hotel Found']

In [54]:
# 8a. Create the output File (CSV)
output_data_file='WeatherPy_Vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [58]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))